## 1. 导入必要的包

In [2]:
import numpy as np
import pandas as pd
import random
import sys
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import funclib

%load_ext autoreload
%autoreload 2

## 2. 加载数据

In [3]:
train = pd.read_hdf('./data/train.h5',key='data')
test = pd.read_hdf('./data/test.h5',key='data')
head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
head = head + ['ec_label','ec_appears']
train.columns = head
test.columns = head

## 3. 同源比对

In [43]:
res_data=funclib.getblast(train,test)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


In [51]:
# 匹配查询结果
id_map_ec = train[['id', 'ec_number']].append(test[['id', 'ec_number']],ignore_index=True)
id_ec_dict = {v: k for v,k in zip( id_map_ec.id, id_map_ec.ec_number)} 
res_data['is_ec_match']=res_data.apply(lambda x: (id_ec_dict.get(x['id'])== id_ec_dict.get(x['sseqid'])), axis=1)

# 输出比对结果
funclib.evaluateBlast(res_data,train,test, 'ec_number')

Total query records are: 18868
Matched records are: 16026
Accuracy: 0.6345134619461522(11972/18868)
Pricision: 0.7470360663921128(11972/16026)
Recall: 0.84937460250159(16026/18868)


## 4. 机器学习EC号预测
### 4.1 onehot + 机器学习

In [58]:
from sklearn import datasets
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC


In [59]:
iris = datasets.load_iris()

In [60]:
X, y = iris.data, iris.target

In [63]:
 clf = OutputCodeClassifier(LinearSVC(random_state=0), code_size=2, random_state=0)

In [64]:
clf.fit(X, y).predict(X)

/home/shizhenkun/anaconda3/envs/jupyterlab-debugger/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/shizhenkun/anaconda3/envs/jupyterlab-debugger/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/shizhenkun/anaconda3/envs/jupyterlab-debugger/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [3]:
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import train_test_split
import os

In [67]:
iris = load_iris()
x = iris['data']
y = iris['target']
dtc = tree.DecisionTreeClassifier(criterion="entropy")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
clf = dtc.fit(x_train, y_train)
print(clf.predict(x_test))
print(y_test)
dot_data = tree.export_graphviz(clf, out_file=None)



[0 1 2 0 0 2 1 0 0 2 1 1 0 1 2]
[0 1 1 0 0 2 1 0 0 2 1 1 0 1 2]


In [ ]:
trainset = train[['id', 'ec_label','seq', 'seqlength']].reset_index(drop=True)
testset = test[['id', 'ec_label','seq', 'seqlength']].reset_index(drop=True)

MAX_SEQ_LENGTH = 1000 #定义序列最长的长度
trainset.seq = trainset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))
testset.seq = testset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))

f_train = funclib.dna_onehot(trainset) #训练集编码
f_test = funclib.dna_onehot(testset) #测试集编码

train_full = pd.concat([trainset, f_train], axis=1, join='inner' ) #拼合训练集
test_full = pd.concat([testset, f_test], axis=1, join='inner' )    #拼合测试集

X_train = np.array(train_full.iloc[:,4:])
X_test = np.array(test_full.iloc[:,4:])
Y_train = np.array(train_full.ec_label.astype('int'))
Y_test = np.array(test_full.ec_label.astype('int'))

NameError: name 'trainset' is not defined

In [6]:
dtc = tree.DecisionTreeClassifier(criterion="entropy")
clf = dtc.fit(X_train, Y_train)
res = clf.predict(X_test)
# print(y_test)
# dot_data = tree.export_graphviz(clf, out_file=None)

In [81]:
Y_test

array([2473,  639, 1791, ..., 3493, 2861, 2608])

In [7]:
aa = pd.DataFrame()
aa['ground_truth'] = Y_test
aa['pred'] = res

In [88]:
len(res)

18868

In [87]:
len(Y_test)

18868

In [8]:
aa['iscorrect']= aa.apply(lambda x: x.ground_truth == x.pred, axis=1)

In [9]:
aa.iscorrect.sum()

1629

In [12]:
aa[aa.iscorrect]

,ground_truth,pred,iscorrect
2,1791,1791,True
5,2473,2473,True
10,1791,1791,True
15,1791,1791,True
17,1791,1791,True
...,...,...,...
18756,3493,3493,True
18757,3493,3493,True
18801,337,337,True
18817,2464,2464,True


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score


In [ ]:
xgboost_clf = XGBClassifier(tree_method='gpu_hist',gpu_id=0,min_child_weight=6,max_depth=15, objective='multi:softmax',num_class=5, n_jobs=-2)
print("-" * 60)
print("xgboost模型：", xgboost_clf)
clf = xgboost_clf.fit(X_train, Y_train)
res = clf.predict(X_test)

------------------------------------------------------------
xgboost模型： XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=15,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-2, num_class=5, num_parallel_tree=None,
              objective='multi:softmax', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)


/home/shizhenkun/anaconda3/envs/jupyterlab-debugger/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:03:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
